In [1]:
#general operations
import time
import os

#data processing
import pandas as pd
import numpy as np

#data visualisation
import matplotlib.pyplot as plt

#standard AWS operations
from sagemaker import get_execution_role
import boto3

import tensorflow as tf

In [23]:
bucket = 'see-ml-storage'
file_name = 'all-userItemInteractions-until-2019-01-13-01-modified.csv'

In [24]:
data_location = 's3://{}/{}'.format(bucket, file_name)

In [25]:
df = pd.read_csv(data_location)

In [26]:
df.shape

(22378, 3)

In [27]:
len(df['USER_ID'].unique())

11859

In [28]:
len(df['ITEM_ID'].unique())

10000

#### Remove Useless Data

In [29]:
df_count=df.groupby('ITEM_ID').count().sort_values('USER_ID', ascending=False)

In [30]:
item_id_keep=df_count[df_count['USER_ID']>=5].index

In [31]:
index_keep=df[df['ITEM_ID'].apply(lambda x: True if x in item_id_keep else False)].index

In [32]:
df=df.iloc[index_keep]

In [33]:
df.shape

(8790, 3)

In [34]:
len(df['USER_ID'].unique())

5381

In [35]:
len(df['ITEM_ID'].unique())

666

#### Check Duplicates

In [32]:
df_clean=df.copy()
df_clean=df_clean[['USER_ID','ITEM_ID']]

In [34]:
sum(df_clean.duplicated())

1478

In [35]:
df_clean.drop_duplicates(inplace=True)

In [36]:
df_clean.head()

,USER_ID,ITEM_ID
2,016434746fb20012218b0e90e1bf1c065002c05d0093c,2288085
3,01667aab565c001fe236ce4ff8df02075005306d00fb8,2254093
6,01667a918deb0014fff4d372d56f03073001606b00978,2995200
8,01667a993d48000243cc54ad6ce40009b008209300408,3429569
33,01667ab66085009acf9f989d801803072004c06a0086e,673120


In [37]:
from tqdm import tqdm_notebook as tqdm

In [38]:
count=[]

for _,row in tqdm(df_clean.iterrows(), total=df_clean.shape[0]):
    count.append(sum((df['ITEM_ID']==row['ITEM_ID']) & (df['USER_ID']==row['USER_ID'])))

In [39]:
df_clean['COUNT']=count

In [41]:
df_clean.reset_index(inplace=True, drop=True)

In [44]:
df_clean.sort_values('COUNT', ascending=False)

,USER_ID,ITEM_ID,COUNT
2764,0166874021190001b727595e884503072001606a009dc,3420290,29
2763,0166874021190001b727595e884503072001606a009dc,3420293,27
2765,0166874021190001b727595e884503072001606a009dc,3420265,20
52,0164348342e50022b3379f0fc19c03072004d06a007e8,3327504,18
20,016661468d0300554edd7dbbb9dc0307200fa06a00bd0,658912,17
3095,0166a97240bd00486727915926bc01075005306d00718,3161428,17
236,01667bceb5b30008ba8f6ae2f90a1b06e001606600bd0,2805925,16
638,016434b5de3300aff0233de7a76004071008c06900bd0,3334728,16
28,0164f880c6bd0014d32337e423af01081007107900978,3404685,14
82,0165a787c9090001a2db1e319a9a1b06d00810650093c,2980739,14
